<a href="https://colab.research.google.com/github/gtbnhyujmj/-Reference-BingX_Market-Data/blob/main/Mark_Price_and_Funding_Rate_%5BFormal%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 標記價格與資金費率 (Mark Price and Funding Rate)
# GET /openApi/swap/v2/quote/premiumIndex
# 不需要 API 金鑰簽名

In [ ]:
import time
import requests
import hmac
from hashlib import sha256

In [ ]:
APIURL = "https://open-api.bingx.com"  # API 基本網址
APIKEY = ""  # 使用者 API 金鑰，需要填入有效金鑰
SECRETKEY = ""  # 使用者秘密金鑰，需要填入有效密鑰

In [ ]:
def demo():
    payload = {}  # GET 請求無請求體
    path = '/openApi/swap/v2/quote/premiumIndex'  # API 路徑：取得標記價格與資金費率
    method = "GET"  # 請求方法

    paramsMap = {
        "symbol": "BTC-USDT"  # 交易對符號，必須包含連字號，如 BTC-USDT
    }

    paramsStr = parseParam(paramsMap)  # 將參數字典轉為 URL 查詢字串並加上 timestamp
    return send_request(method, path, paramsStr, payload)  # 發送請求並回傳結果

In [ ]:
def get_sign(api_secret, payload):
    # 以 HMAC-SHA256 算法，對參數字串產生簽名

    signature = hmac.new(api_secret.encode("utf-8"), payload.encode("utf-8"), digestmod=sha256).hexdigest()
    print("sign=" + signature)  # 輸出簽名供調試用

    return signature  # 回傳簽名字串

In [ ]:
def send_request(method, path, urlpa, payload):

    # 組合完整 URL，包含簽名參數
    url = "%s%s?%s&signature=%s" % (APIURL, path, urlpa, get_sign(SECRETKEY, urlpa))
    print(url)  # 輸出完整請求 URL 方便除錯

    headers = {
        'X-BX-APIKEY': APIKEY,  # HTTP 標頭帶入 API 金鑰
    }

    # 發送 HTTP 請求，包含方法、URL、標頭與載荷
    response = requests.request(method, url, headers=headers, data=payload)
    return response.text  # 回傳 API 回應的文字內容

In [ ]:
def parseParam(paramsMap):
    sortedKeys = sorted(paramsMap)  # 參數鍵排序，確保簽名一致

    # 組合成 URL 查詢字串，如 key1=value1&key2=value2
    paramsStr = "&".join(["%s=%s" % (x, paramsMap[x]) for x in sortedKeys])

    if paramsStr != "":
        # 若有參數，加上 timestamp 時間戳（毫秒）
        return paramsStr+"&timestamp="+str(int(time.time() * 1000))
    else:
        # 若無參數，只有 timestamp
        return paramsStr+"timestamp="+str(int(time.time() * 1000))

In [ ]:
if __name__ == '__main__':
    print("demo:", demo())  # 執行範例函式並印出結果